## About

*In this notebook, we implement the Heston pricer, the different differentials of the normalised forward put price w.r.t inputs as well as the generator used to build the datasets utilised to train, validate, and test the neural networks.*

---
**Date: September 2022**  

**Author:** Abir Sridi ([Imperial College](https://www.imperial.ac.uk) | [MSc AI](https://www.imperial.ac.uk/study/pg/computing/artificial-intelligence/))


**Contact:** a.sridi@imperial.ac.uk

# <strong> 1. Prerequisites</strong>

## <Strong><font color='green'>1.1. Imports</font></Strong>: Python packages needed

In [ ]:
!pip install scipy==1.7
!pip install smt # for Latin Hypercube sampling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 28.5 MB 1.4 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.7.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 252 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51 kB 765 kB/s 
     |████████████████████████████████| 3.2 MB 56.2 MB/s 
     |███████████████████████████

In [ ]:
# Various packages
import os
import sys
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import time
import datetime
from google.colab import files # to save the dataset into CSV files, to save figures
np.random.seed(42) # set a seed for the random generator
from smt.sampling_methods import LHS # to perform Latin Hypercube Sampling (LHS)

# scipy
from scipy.integrate import quad_vec  # quad_vec allows to compute integrals accurately
from scipy.stats import norm
from scipy.stats import qmc # to perform Latin Hypercube Sampling (LHS) 



# <strong> 2. European Call & Put prices </strong>

## <Strong><font color='green'>2.1. Implement the characteristic function</font></Strong>

In [ ]:
def beta_function(u, tau, sigma, rho, kappa):
    return kappa - 1j * u * sigma * rho

def alpha_hat_function(u):
    return -0.5 * u * (u + 1j)

def d_function(u, tau, sigma, rho, kappa):
    gamma = 0.5 * sigma**2
    beta = beta_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)
    return np.sqrt(beta**2 - 4 * alpha_hat * gamma)

def g_function(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    return (beta - d) / (beta + d)

def A_function(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    common_term = np.exp(-d*tau)
    A_u = (kappa * theta / (sigma**2)) * ((beta-d)*tau - 2*np.log((g*common_term-1) / (g-1)))    
    return A_u

def B_function(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    common_term = np.exp(-d*tau)
    B_u = ((beta-d) / (sigma**2)) * ((1 - common_term) / (1 - g*common_term))
    return B_u


In [ ]:
def heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0):

    beta = beta_function(u, tau, sigma, rho, kappa)    
    #alpha_hat = alpha_hat_function(u)    
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    common_term = np.exp(-d*tau)
    A = A_function(u, tau, theta, sigma, rho, kappa)
    B = B_function(u, tau, sigma, rho, kappa)

    return np.exp(A + B * v0)

## <Strong><font color='green'>2.2. Perform numerical integration</font></Strong>: using scipy integrate quad_vec function

In [ ]:
def integral_price(m, tau, theta, sigma, rho, kappa, v0):
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m)*heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return integ

## <Strong><font color='green'>2.3. Calculate European Call price </font></Strong>

In [ ]:
def call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0):
    m = np.log(S0/k) + r*tau #log-moneyness forward
    integ = integral_price(m, tau, theta, sigma, rho, kappa, v0)  
    price = S0 - k * np.exp(-r*tau) * integ  /np.pi
         
    return price

## <Strong><font color='green'>2.4. Calculate European put price </font></Strong>: using the call-put parity relation


In [ ]:
def put_price(k, tau, S0, r, theta, sigma, rho, kappa, v0):
    price = call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)- S0 + k * np.exp(-r*tau)
    return price

## <Strong><font color='green'>2.5. Calculate the Normalised Forward Put Price $\hat{P}$ </font></Strong>

$\hat{P}(t,S_t,v_t) = \frac{e^{rT}}{K} P(t,S_t,v_t)$

In [ ]:
def norm_forw_put_price(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r*tau #log-moneyness forward
    integ = integral_price(m, tau, theta, sigma, rho, kappa, v0)
    return 1 - (1/np.pi) * integ

### Sanity check: $\hat{P}$ calculation

In [ ]:

def test_p_hat():
    k = 100.0 # strike
    S0 = 100.0 # initial asset price
    lm = np.log(S0/k)
    r=0.03
    tau=1.
    theta=0.0398
    sigma=0.3
    rho=-0.5711
    kappa=1.5768
    v0=0.1

    call = call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)
    put = put_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)
    p_hat_expected = (np.exp(r*tau)/k) * put
    p_hat = norm_forw_put_price(lm, r, tau, theta, sigma, rho, kappa, v0)

    np.testing.assert_almost_equal(p_hat, p_hat_expected, decimal=5, err_msg='', verbose=True)    

test_p_hat()  


## <Strong><font color='green'>2.6. Black-Scholes as a particular case </font></Strong>
The Black-Scholes model is embedded inside the Heston model. Therefore, the European option Heston price under specific parameter values will be the Black-Scholes price. To check our Heston prices implementation, we will price European calls and put options under Black-Scholes and the Heston model with the parameter values leading to the Black-Scholes prices. We have to obtain comparable prices. Note that we cannot substitute $σ = 0$ into the Heston pricing functions because that will lead to division by zero. We will take small value of $\sigma$ (e.g $10^{-05}$).

In [ ]:
def d1(k, tau, S0, r, sigma):
    return(np.log(S0/k)+(r+((sigma**2)/2.))*tau)/(sigma*np.sqrt(tau))

def d2(k, tau, S0, r, sigma):
    return d1(k,tau, S0, r, sigma) - sigma*np.sqrt(tau)

# Black-Scholes Call price 
def call_bs(k, tau, S0, r, sigma):
    return S0*norm.cdf(d1(k, tau, S0, r, sigma))-k*np.exp(-r*tau)*norm.cdf(d2(k, tau, S0, r, sigma))

# Black-Scholes Put price using the call-put parity relation
def put_bs(k, tau, S0, r, sigma):
    return call_bs(k,tau, S0, r, sigma) + k*np.exp(-r*tau) - S0    


### 2.6.1 First case: small $σ$, $θ = v_0$ and $σ_{BS} = \sqrt{v_0}$

In [ ]:
def test_call_heston():
    k=100.0
    tau=1.
    S0=100.
    r=0.03
    theta=0.1
    sigma=0.00001
    #sigma=0.000001

    rho=-0.5711
    kappa=1.5768
    #kappa=0.001
    #kappa=3 
    #kappa=1.5
    v0=0.1

    sigma_BS = np.sqrt(v0)
    call_heston = call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)
    call_BS = call_bs(k, tau, S0, r, sigma_BS)
    #print(call_heston)
    #print(call_BS)

    np.testing.assert_almost_equal(call_heston, call_BS, decimal=5, err_msg='', verbose=True)    

test_call_heston()

In [ ]:
def test_put_heston():
    k=100.0
    tau=1.
    S0=100.
    r=0.03
    theta=0.1
    sigma=0.00001
    rho=-0.5711
    kappa=1.5768
    v0=0.1

    sigma_BS = np.sqrt(v0)
    put_heston = put_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)
    put_BS = put_bs(k, tau, S0, r, sigma_BS)

    np.testing.assert_almost_equal(put_heston, put_BS, decimal=5, err_msg='', verbose=True)    

test_put_heston()

### 2.6.2 Second case: Small $\sigma$ and 
\begin{equation}
\sigma^2_{BS} = \theta + \frac{1-e^{-\kappa \tau}}{\kappa \tau} (v_0-\theta)
\end{equation}

In [ ]:
def test_call_heston():
    k=100.0
    tau=1.
    S0=100.
    r=0.03
    theta=0.1
    sigma=0.00001
    rho=-0.5711
    kappa=1.5768
    v0=0.1

    coeff = (1-np.exp(-kappa*tau))/(kappa*tau)
    sigma_BS = np.sqrt(theta+(v0-theta)*coeff)
    
    call_heston = call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)
    call_BS = call_bs(k, tau, S0, r, sigma_BS)

    np.testing.assert_almost_equal(call_heston, call_BS, decimal=5, err_msg='', verbose=True)    

test_call_heston()

In [ ]:
def test_put_heston():
    k=100.0
    tau=1.
    S0=100.
    r=0.03
    theta=0.1
    sigma=0.00001
    rho=-0.5711
    kappa=1.5768
    v0=0.1

    coeff = (1-np.exp(-kappa*tau))/(kappa*tau)
    sigma_BS = np.sqrt(theta+(v0-theta)*coeff)
    
    put_heston = put_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)
    put_BS = put_bs(k, tau, S0, r, sigma_BS)

    np.testing.assert_almost_equal(put_heston, put_BS, decimal=5, err_msg='', verbose=True)    

test_put_heston()

# <strong> 3. Differential of the Normalised Forward Put Price w.r.t inputs </strong> 

In [ ]:
def integral_delta(m, tau, theta, sigma, rho, kappa, v0):
    integrand = (lambda u: 
        np.real((1j*u + 0.5) * np.exp((1j*u + 0.5)*m)*heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return integ

In [ ]:
def term_interm_theta(u, tau, sigma, rho, theta, kappa, v0):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)

    term1 = kappa*theta * (beta-d+2*((d*g*np.exp(-d*tau)) / ((g*np.exp(-d*tau))-1))) /(sigma**2)
    term2 = v0*(beta-d)*d * np.exp(-d*tau)*(1-g)/((sigma**2)*(1-g*np.exp(-d*tau))**2)
    term = term1 + term2

    return term    

In [ ]:
def integral_differential_phi_tau(m, tau, sigma, rho, theta, kappa, v0):

    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m)*heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0) * 
                term_interm_theta(u - 0.5j, tau, sigma, rho, theta, kappa, v0))  /(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return integ

## <Strong><font color='green'>3.1. $\frac{\partial \hat{P}}{\partial \theta}$ </font></Strong>

In [ ]:
def differential_wrt_theta(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r * tau     #log-moneyness forward
    if theta == 0.:
        theta = 0.00000001
    
    integrand = (lambda u: 
        np.real((1/theta) * np.exp((1j*u + 0.5)*m)* A_function(u - 0.5j, tau, theta, sigma, rho, kappa) * heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ

In [ ]:
def test_differential_wrt_theta():    
    delta_theta = 0.00000001
    theta0 = 0.7
    m = 1. + 0.03*5.
    p_hat_deltathetaplustheta0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=delta_theta+theta0,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    p_hat_theta0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=theta0,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)

    differential_formula = differential_wrt_theta(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltathetaplustheta0 - p_hat_theta0)/delta_theta 
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=8, err_msg='', verbose=True)  

test_differential_wrt_theta()
  

## <Strong><font color='green'>3.2. $\frac{\partial \hat{P}}{\partial lm}$ </font></Strong>
lm is the log-moneyness

In [ ]:
def differential_wrt_lm(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r * tau #log-moneyness forward
    res = - (1/np.pi) * integral_delta(m, tau, theta, sigma, rho, kappa, v0)
    return res

In [ ]:
def test_differential_wrt_lm():    
    delta_lm = 0.00000001
    lm0 = 1.

    p_hat_deltampluslm0 = norm_forw_put_price(lm=delta_lm+lm0, r=0.03,tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    p_hat_lm0 = norm_forw_put_price(lm=lm0, r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)

    differential_formula = differential_wrt_lm(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltampluslm0 - p_hat_lm0)/delta_lm
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=8, err_msg='', verbose=True)   

test_differential_wrt_lm()
  

## <Strong><font color='green'>3.3. $\frac{\partial \hat{P}}{\partial v_o}$ </font></Strong>

In [ ]:
def differential_wrt_v0(lm, r, tau, theta, sigma, rho, kappa, v0):    
    m = lm + r * tau #log-moneyness forward
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m)* B_function(u - 0.5j, tau, sigma, rho, kappa) * heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ

In [ ]:
def test_differential_wrt_v0():    
    delta_v0 = 0.00000001
    v0 = 0.25

    p_hat_deltav0plusv0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=delta_v0+v0)
    p_hat_v0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=v0)

    differential_formula = differential_wrt_v0(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltav0plusv0 - p_hat_v0)/delta_v0
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=7, err_msg='', verbose=True)   

test_differential_wrt_v0()

## <Strong><font color='green'>3.4. $\frac{\partial \hat{P}}{\partial \tau}$ </font></Strong>

In [ ]:
def differential_wrt_tau(lm, r, tau, theta, sigma, rho, kappa, v0):    
    m = lm + r * tau #log-moneyness forward
    integ1 = integral_delta(m, tau, theta, sigma, rho, kappa, v0)
    integ2 = integral_differential_phi_tau(m, tau, sigma, rho, theta, kappa, v0)   

    return (-1/np.pi) * (r * integ1 + integ2)

In [ ]:
def test_differential_wrt_tau():    
    delta_tau = 0.00000001
    tau0 = 5.
    p_hat_deltatauplustau0 = norm_forw_put_price(lm=1., r=0.03, tau=delta_tau+tau0, theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    p_hat_tau0 = norm_forw_put_price(lm=1., r=0.03, tau=tau0, theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)

    differential_formula = differential_wrt_tau(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltatauplustau0 - p_hat_tau0)/delta_tau
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=7, err_msg='', verbose=True)   

test_differential_wrt_tau()

## <Strong><font color='green'>3.5. $\frac{\partial \hat{P}}{\partial \kappa}$ </font></Strong>

In [ ]:
def differential_A_kappa(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    A_u = A_function(u, tau, theta, sigma, rho, kappa)

    res = A_u/kappa - (kappa * theta / (d * (sigma**2))) * (-d*tau+tau*beta+4*g/(g-1)+2*g*np.exp(-d*tau)*(2+tau*beta)/(1-g*np.exp(-d*tau)))
    return res

In [ ]:
def differential_B_kappa(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    B_u = B_function(u, tau, sigma, rho, kappa) 

    res = -B_u/d + (1/d) * (tau*beta*np.exp(-d*tau)*B_u/(1-np.exp(-d*tau)) - g*np.exp(-d*tau)*(2+tau*beta)*B_u/(1-g*np.exp(-d*tau)))
    return res

In [ ]:
def differential_phi_kappa(u, tau, theta, sigma, rho, kappa, v0):
    return heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0) * (differential_A_kappa(u, tau, theta, sigma, rho, kappa) +
                                                                         v0 * differential_B_kappa(u, tau, theta, sigma, rho, kappa))


In [ ]:
def differential_wrt_kappa(lm, r, tau, theta, sigma, rho, kappa, v0): 
    m = lm + r*tau   #log-moneyness forward   
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m) * differential_phi_kappa(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ   

In [ ]:
def test_differential_wrt_kappa():    
    delta_kappa = 0.00000001
    kappa0 = 0.15
    p_hat_deltakappapluskappa0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=delta_kappa+kappa0, v0=0.25)
    p_hat_kappa0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=kappa0, v0=0.25)

    differential_formula = differential_wrt_kappa(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltakappapluskappa0 - p_hat_kappa0)/delta_kappa
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=7, err_msg='', verbose=True)   

test_differential_wrt_kappa()

## <Strong><font color='green'>3.6. $\frac{\partial \hat{P}}{\partial \rho}$ </font></Strong>

In [ ]:
def differential_A_rho(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)

    res = (kappa*theta*1j*u/(d*sigma)) * (tau*(beta-d)-2*g*(np.exp(-d*tau)*(2+tau*beta)/(g*np.exp(-d*tau)-1)-2/(g-1)))
    return res 

In [ ]:
def differential_B_rho(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    B_u = B_function(u, tau, sigma, rho, kappa) 

    res = (1j*u*sigma*B_u/d) * (1+np.exp(-d*tau)*(-tau*beta/(1-np.exp(-d*tau))+g*(2+tau*beta)/(1-g*np.exp(-d*tau))))
    return res

In [ ]:
def differential_phi_rho(u, tau, theta, sigma, rho, kappa, v0):
    return heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0) * (differential_A_rho(u, tau, theta, sigma, rho, kappa) +
                                                                         v0 * differential_B_rho(u, tau, sigma, rho, kappa))

In [ ]:
def differential_wrt_rho(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r*tau  #log-moneyness forward
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m) * differential_phi_rho(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ   

In [ ]:
def test_differential_wrt_rho():    
    delta_rho = 0.00000001
    rho0 = -0.6
    p_hat_deltarhoplusrho0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=delta_rho+rho0, kappa=0.15, v0=0.25)
    p_hat_rho0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=0.7, rho=rho0, kappa=0.15, v0=0.25)

    differential_formula = differential_wrt_rho(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltarhoplusrho0 - p_hat_rho0)/delta_rho
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=7, err_msg='', verbose=True)   

test_differential_wrt_rho()

## <Strong><font color='green'>3.7. $\frac{\partial \hat{P}}{\partial \sigma}$ </font></Strong>

In [ ]:
def differential_ln_sigma(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)    

    res= ((2j*u*rho*((beta**2)-(d**2))+4*beta*alpha_hat*sigma)*((g-1)*np.exp(-d*tau)-g*np.exp(-d*tau)+1)+
          (g-1)*np.exp(-d*tau)*tau*g*((beta+d)**2)*(1j*u*rho*beta+2*alpha_hat*sigma))/(d*((beta+d)**2)*(g-1)*(g*np.exp(-d*tau)-1))
    return res

In [ ]:
def differential_A_sigma(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    A_u = A_function(u, tau, theta, sigma, rho, kappa)
    diff_ln_sigma = differential_ln_sigma(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)

    res = -2*A_u/sigma + kappa*theta/(sigma**2)*(-1j*u*tau*rho+tau*(1j*u*rho*beta+2*alpha_hat*sigma)/d - 2*diff_ln_sigma)
    return res 

In [ ]:
def differential_quotient_sigma(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)

    res = ((np.exp(-d*tau)*(1j*u*rho*beta+2*alpha_hat*sigma))*(-tau*((beta+d)**2)*(1-g*np.exp(-d*tau))+(1-np.exp(-d*tau))*(2*beta+tau*g*((beta+d)**2)))
     - 2j*u*rho*(d**2)*np.exp(-d*tau)*(1-np.exp(-d*tau))) / (d*((beta+d)**2)*((1-g*np.exp(-d*tau))**2))
    return res

In [ ]:
def differential_B_sigma(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)
    
    diff_quot_sigma = differential_quotient_sigma(u, tau, sigma, rho, kappa)
    res = ((1-np.exp(-d*tau))*(sigma*(-1j*u*rho*d+1j*u*rho*beta+2*alpha_hat*sigma)-2*d*(beta-d))) / (d*(sigma**3)*(1-g*np.exp(-d*tau))) + (beta-d)/(sigma**2)*diff_quot_sigma
    return res

In [ ]:
def differential_phi_sigma(u, tau, theta, sigma, rho, kappa, v0):
    return heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0) * (differential_A_sigma(u, tau, theta, sigma, rho, kappa) +
                                                                         v0 * differential_B_sigma(u, tau, sigma, rho, kappa))

In [ ]:
def differential_wrt_sigma(lm, r, tau, theta, sigma, rho, kappa, v0):  
    m = lm + r*tau  #log-moneyness forward      
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m) * differential_phi_sigma(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ   

In [ ]:
def test_differential_wrt_sigma():    
    delta_sigma = 0.00000001
    sigma0 = 0.7
    p_hat_deltasigmaplussigma0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=delta_sigma+sigma0, rho=-0.6, kappa=0.15, v0=0.25)
    p_hat_sigma0 = norm_forw_put_price(lm=1., r=0.03, tau=5., theta=0.7,  sigma=sigma0, rho=-0.6, kappa=0.15, v0=0.25)

    differential_formula = differential_wrt_sigma(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltasigmaplussigma0 - p_hat_sigma0)/delta_sigma
    
    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=8, err_msg='', verbose=True)   

test_differential_wrt_sigma()

## <Strong><font color='green'>3.8. $\frac{\partial \hat{P}}{\partial r}$ </font></Strong>

In [ ]:
def differential_wrt_r(lm, r, tau, theta, sigma, rho, kappa, v0):
    diff_wrt_lm = differential_wrt_lm(lm, r, tau, theta, sigma, rho, kappa, v0)
    return tau * diff_wrt_lm

In [ ]:
def test_differential_wrt_r():
    delta_r = 0.00000001
    r0 = 0.03
    p_hat_deltarplusr0 = norm_forw_put_price(lm=1., r=delta_r+r0, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    p_hat_r0 = norm_forw_put_price(lm=1., r=r0, tau=5., theta=0.7,  sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)

    differential_formula = differential_wrt_r(lm=1., r=0.03, tau=5., theta=0.7, sigma=0.7, rho=-0.6, kappa=0.15, v0=0.25)
    differential_finite_difference = (p_hat_deltarplusr0 - p_hat_r0)/delta_r

    np.testing.assert_almost_equal(differential_formula, differential_finite_difference, decimal=8, err_msg='', verbose=True)   

test_differential_wrt_r()    

# <strong> 4. Data generation </strong>

## <Strong><font color='green'> 4.1. Inputs generation </font></Strong> 

####Latin Hypercube Sampling

$\theta$, $\sigma$, $\kappa$, $\rho$ parameters, $v_0$, Log moneyness $lm=\ln(S_0/K)$, the time to maturity $\tau$ and $r$ will be sampled via Latin Hypercube Sampling (LHS) technique. 

In the following, we will use two modules to perform the LHS: *scipy.stats.qmc* and *smt.sampling_methods.LHS*. We will calculate the exact time every program takes to sample 10 points and keep the one with less time to generate the inputs.

In [ ]:
# LHS using smt.sampling_methods.LHS

# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)

start_time = time.clock()

bounds = np.array([[-2.,2.], [-0.01,0.10], [0.05,20], [0.0,1.], [0.1,2.], [-0.90,0.0], [0.005,3.], [0.,1.]])
sampling = LHS(xlimits=bounds, random_state=42)

n_samples = 10
samples = sampling(n_samples)

print("The exact time  the program takes to finish running: %s seconds " % (time.clock() - start_time))
#print(samples)

The exact time  the program takes to finish running: 0.0020979999999966026 seconds 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


In [ ]:
# LHS using scipy.stats.qmc
# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)

start_time = time.clock()  

sampler = qmc.LatinHypercube(d=8, seed=42) # 6 variables
samples = sampler.random(n=10) # n samples

lower_bounds = [-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.] 
upper_bounds = [2., 0.10, 20., 1.0, 2., 0.0, 3., 1.]
inputs_array = qmc.scale(samples, lower_bounds, upper_bounds)

print("The exact time  the program takes to finish running: %s seconds " % (time.clock() - start_time))
#print(inputs_array)

The exact time  the program takes to finish running: 0.0017220000000008895 seconds 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


Note that *scipy.stats.qmc* module takes less time than *smt.sampling_methods.LHS*, therefore, inputs generation will be performed using *scipy.stats.qmc* module. 

We will generate 100 000 (100K) samples. 

In [ ]:
def generate_inputs(d, n, lower_bounds, upper_bounds, feller=False):
    
    sampler = qmc.LatinHypercube(d, seed=42) # d variables    
    samples = sampler.random(n) # n samples
    inputs_array = qmc.scale(samples, lower_bounds, upper_bounds)
    
    if feller:
        #Selecting samples that satisfy the Feller condition        
        inputs_array = inputs_array[np.where(2*inputs_array[:,6]*inputs_array[:,3] > (inputs_array[:,4])**2)]

    return inputs_array

## <Strong><font color='green'> 4.2. Labels and their differentials w.r.t inputs generation </font></Strong> 

In [ ]:
def generate_labels_difflabels(inputs_array):
    start_time = time.clock()

    # array containing labels and its differentials
    labels_difflabels_array = np.zeros((inputs_array.shape[0],inputs_array.shape[1]+1))

    for row in range(inputs_array.shape[0]):

        lm = inputs_array[row][0] 
        r = inputs_array[row][1]
        tau = inputs_array[row][2]
        theta = inputs_array[row][3]
        sigma = inputs_array[row][4]
        rho = inputs_array[row][5]      
        kappa = inputs_array[row][6]
        v0 = inputs_array[row][7]    

        p_hat = norm_forw_put_price(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_lm = differential_wrt_lm(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_r = differential_wrt_r(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_tau = differential_wrt_tau(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_theta = differential_wrt_theta(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_sigma = differential_wrt_sigma(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_rho = differential_wrt_rho(lm, r, tau, theta,sigma, rho, kappa, v0)  
        diff_wrt_kappa = differential_wrt_kappa(lm, r, tau, theta,sigma, rho, kappa, v0)    
        diff_wrt_v0 = differential_wrt_v0(lm, r, tau, theta,sigma, rho, kappa, v0)             

        labels_difflabels_array[row][0] = p_hat 
        labels_difflabels_array[row][1] = diff_wrt_lm
        labels_difflabels_array[row][2] = diff_wrt_r
        labels_difflabels_array[row][3] = diff_wrt_tau
        labels_difflabels_array[row][4] = diff_wrt_theta
        labels_difflabels_array[row][5] = diff_wrt_sigma
        labels_difflabels_array[row][6] = diff_wrt_rho
        labels_difflabels_array[row][7] = diff_wrt_kappa
        labels_difflabels_array[row][8] = diff_wrt_v0          

    print("The exact time  the program takes to finish running:", time.strftime('%H:%M:%S', time.gmtime(time.clock() - start_time))) 
    return labels_difflabels_array


## <Strong><font color='green'> 4.3. Dataset generation </font></Strong> 

In [ ]:
def generate_data(d, n, lower_bounds, upper_bounds, feller, first_samples=100000):

    inputs_array = generate_inputs(d, n, lower_bounds, upper_bounds, feller)
    if feller:
        #extract first 100000 samples
        #inputs_array=inputs_array[0:100000, :]
        inputs_array=inputs_array[0:first_samples, :]

    labels_difflabels_array = generate_labels_difflabels(inputs_array)

     
    # Convert the inputs array to a dataframe
    df_inputs = pd.DataFrame(inputs_array, columns=['lm', 'r', 'tau', 'theta', 'sigma', 'rho', 'kappa', 'v0'])

    # Convert the array containing labels and its differentials to a dataframe
    df_labels_difflabels = pd.DataFrame(labels_difflabels_array, columns=["p_hat", "diff wrt lm", "diff wrt r", "diff wrt tau", "diff wrt theta",
                                                "diff wrt sigma", "diff wrt rho", "diff wrt kappa", "diff wrt v0"])
    # put the whole data into a dataframe
    df_dataset = pd.concat([df_inputs, df_labels_difflabels], axis=1)     
    
        
    return df_dataset

### 4.3.1 Data generation when the Feller condition is breached

The following algorithm takes 10 hours, 34 minutes and 24 seconds.

In [ ]:
# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)
#lower_bounds = [-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.] 
#upper_bounds = [2., 0.10, 20., 1.0, 2., 0.0, 3., 1.]

df_nofeller = generate_data(d=8, n=100000,                                      
                                     lower_bounds=[-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.], 
                                     upper_bounds=[2., 0.10, 20., 1.0, 2., 0.0, 3., 1.],
                                     feller=False)

# # Saving datasets into CSV files
df_nofeller.to_csv('dataset_100K_nofeller.csv', sep=',', index=False)
files.download('dataset_100K_nofeller.csv') 

### 4.3.2 Data generation when the Feller condition is satisfied 

The following algorithm takes 9 hours, 14 minutes and 51 seconds.

In [ ]:
# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)
#lower_bounds = [-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.] 
#upper_bounds = [2., 0.10, 20., 1.0, 2., 0.0, 3., 1.]

df_feller = generate_data(d=8, n=100000, 
                                   lower_bounds=[-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.], 
                                   upper_bounds=[2., 0.10, 20., 1.0, 2., 0.0, 3., 1.],
                                   feller=True)

# # Saving datasets into CSV files
df_feller.to_csv('dataset_100K_feller.csv', sep=',', index=False)
files.download('dataset_100K_feller.csv')  


### 4.3.3 Check that the samples in the dataframe *df_feller* satisfy the Feller condition

In [ ]:
def test_generate_data():    

    df_feller = df_nofeller[2*df_nofeller['kappa']*
                                              df_nofeller['theta'] > (df_nofeller['sigma'])**2]
    df_feller = df_feller.reset_index(drop=True)
    assert df_feller.equals(df_feller) == True

test_generate_data()    